In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..\..'))
if module_path not in sys.path:
    sys.path.insert(0, module_path)

In [73]:
from funcs import *

## Read Data + Save

In [4]:
human_data(file='data/BIOGRID-ORGANISM-Homo_sapiens-4.4.204.tab3.txt',
           save_file='data/homo_preprocess.tsv')

In [5]:
disease_data(file='data/curated_gene_disease_associations.tsv',
             disease_id='C0003873')

## Data Overview

In [6]:
data_overview(human_file='data/homo_preprocess.tsv',
              disease_file='data/diseaseC0003873.tsv')

Number of genes associated with the desease: 174
Classes of the desease: ['C17;C05;C20']
Number of genes present in the interactome: 173
Largest connected component: 83


## Algorithms

In [ ]:
MCL(human_file='data/homo_preprocess.tsv', start=20, end=27)

In [15]:
DIAMOND(network_file='data/PPI.txt', seed_file='data/seed_genes.txt', n=200, alpha=1, out_file='data/diamond_results.txt')

DIAMOnD(): ignoring 1 of 174 seed genes that are not in the network

 results have been saved to 'data/diamond_results.txt' 



In [61]:
DIABLE(network_file='data/PPI.txt', seed_file='data/seed_genes.txt', n=200, alpha=1, out_file='data/diable_results.txt')

DiaBLE(): ignoring 1 of 174 seed genes that are not in the network

 results have been saved to 'data/diable_results.txt' 



In [91]:
RANDOM_WALK_WITH_RESTART(network_file='data/PPI.txt', seed_file='data/seed_genes.txt', r=0.1, score_thr=0, tol=1e-6)

RANDOM_WALK_WITH_RESTART(): ignoring 1 of 174 seed genes that are not in the network

 results have been saved to 'data/random_walk_wr_results.txt' 



## 10-fold cross validation

## Extended validation